# Amazon Web Scraper

In [95]:
import csv
from bs4 import BeautifulSoup

In [96]:
from selenium import webdriver

# Startup the webdriver

In [98]:
driver = webdriver.Chrome("C:\ChromeDriver\chromedriver_win32\chromedriver")

In [ ]:
# edge
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options = options)

In [99]:
url = 'https://www.amazon.com'
driver.get(url)

In [100]:
def get_url(search_term):
    template = "https://www.amazon.in/s?k={}&ref=nb_sb_noss"
    search_term = search_term.replace(' ','+')
    return template.format(search_term)

In [101]:
url = get_url('Wipro Led Light')
print(url)

https://www.amazon.in/s?k=Wipro+Led+Light&ref=nb_sb_noss


In [102]:
driver.get(url)

# Extract The Collection

In [103]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [104]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

In [105]:
len(results)

60

# Prototype the Record

In [108]:
item = results[0]

In [109]:
atag = item.h2.a

In [110]:
description = atag.text.strip()

In [112]:
url = 'https://www.amazon.com' + atag.get('href')

In [113]:
price_parent = item.find('span','a-price')

In [115]:
price_parent.find('span', 'a-offscreen').text

'₹549'

In [118]:
rating = item.i.text
rating

'4.1 out of 5 stars'

In [123]:
review_count = item.find('span', {'class': 'a-size-base'}).text
review_count

'235'

# Generalize the pattern

In [124]:
def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    price_parent = item.find('span','a-price')
    price_parent.find('span', 'a-offscreen').text
    
    rating = item.i.text
    review_count = item.find('span', {'class': 'a-size-base'}).text
    
    result = (description, price, rating, review_count, url)
    return result

In [125]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    records.append(extract_record(item))

NameError: name 'price' is not defined

In [129]:
def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        price_parent = item.find('span','a-price')
        price = price_parent.find('span', 'a-offscreen').text
        
        
    except AttributeError:
        return
    
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
        
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    return result

In [130]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [131]:
records[0]

('Wipro Brillante B22 30-Watts LED Cool Day Light Lamp (White, Pack of 1)',
 '₹549',
 '4.1 out of 5 stars',
 '235',
 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0554146RL4I4OVY6KDE&url=%2FWipro-Brillante-Watt-Base-White%2Fdp%2FB08H5PRD19%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3DWipro%2BLed%2BLight%26qid%3D1632768654%26sr%3D8-1-spons%26psc%3D1&qualifier=1632768654&id=6078575372808979&widgetName=sp_atf')

In [132]:
for row in records:
    print(row[1])

₹549
₹5,185
₹349
₹255
₹849
₹749
₹599
₹249
₹899
₹499
₹280
₹409
₹624
₹561
₹768
₹282
₹380
₹352
₹379
₹480
₹265
₹785
₹860
₹1,885
₹497
₹210
₹780
₹1,099
₹736
₹1,198
₹1,009
₹530
₹447
₹899
₹130
₹725
₹496
₹658
₹573
₹274
₹624
₹559
₹440
₹229
₹324
₹799
₹288
₹599
₹599
₹869
₹719
₹314
₹690
₹899
₹499
₹453
₹330
₹545
₹500
₹699


# Geeting the Next Page

In [133]:
def get_url(search_term):
    template = "https://www.amazon.in/s?k={}&ref=nb_sb_noss"
    search_term = search_term.replace(' ','+')
    
    # add term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    url += '&page{}'
    
    return url

# Putting It All Together

In [137]:
import csv
from bs4 import BeautifulSoup

def get_url(search_term):
    template = "https://www.amazon.in/s?k={}&ref=nb_sb_noss"
    search_term = search_term.replace(' ','+')
    
    # add term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    url += '&page{}'
    
    return url


def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        price_parent = item.find('span','a-price')
        price = price_parent.find('span', 'a-offscreen').text
        
        
    except AttributeError:
        return
    
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
        
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    return result


def main(search_term):
    
    # Startup the Webdriver
    driver = webdriver.Chrome("C:\ChromeDriver\chromedriver_win32\chromedriver")
    
    record = []
    url = get_url(search_term)
    
    for page in range(1, 21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    
    # Save Data To CSV File
    
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description','Price','Rating','ReviewCount','Url'])
        writer.writerows(records)

In [138]:
main('wipro led light')